# Project: Customer Relationship Prediction
Univ.AI <br>
AI-1 Cohort-5

## Name of the people who worked on the project:


Link to presentation: *link here*

Link to slides: *link here*

Link to poster: *link here*

Any other information:*link here*

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (confusion_matrix, auc, f1_score,roc_auc_score)
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

In [2]:
df = pd.read_csv("./orange_small_train.data", sep = "\t")


In [3]:
churn = pd.read_table('orange_small_train_churn.labels', header = None).loc[:, 0]


In [4]:
churn = pd.Series(churn)
df = df.assign(churn = churn)

In [5]:
s = df.shape[1] 

In [6]:
missing_perc = 20
min_count = int(((100 - missing_perc) / 100) * df.shape[0] + 1)
min_count

40001

In [7]:
#drop all missing and only continous variable
prefered_uniques = 10
# create a list of numeric values and less than prefered uniques categorical columns
fe_unique_columns = list(df.select_dtypes(include='number').columns) + list(
    (df.select_dtypes(include='object').nunique() < prefered_uniques).index[
        df.select_dtypes(include='object').nunique() < prefered_uniques])
# collect only numeric features
df_all_missing_drop_continous = df.dropna( axis=1,thresh=min_count).dropna(axis=0).select_dtypes(include=['number'])#.agg(['nunique','count'])
numeric_col_after_drop = df_all_missing_drop_continous.columns
# create df with categorical variables with less than 10 uniques
df_all_missing_dropped_prefered_unique_cat = df[fe_unique_columns].dropna( axis=1,thresh=min_count).dropna(axis=0)
feature_prefered_columns = list(df_all_missing_dropped_prefered_unique_cat.columns)
# creating dummy variables for the data with  categorical variables
df_encoded = pd.get_dummies(df_all_missing_dropped_prefered_unique_cat,drop_first=True)
df_numeric = df_all_missing_drop_continous
del df_all_missing_drop_continous

In [8]:
df.dropna(axis=1, thresh=min_count).dropna(axis=0).select_dtypes(include='object').agg(['nunique']) < prefered_uniques

,Var192,Var193,Var195,Var196,Var197,Var198,Var199,Var202,Var203,Var204,...,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var226,Var227,Var228
nunique,False,False,False,True,False,False,False,False,True,False,...,False,True,False,False,True,False,True,False,True,False


In [9]:
df.dropna(axis=1, thresh=min_count).dropna(axis=0)['Var196'].unique()


array(['1K8T', 'z3mO', 'mKeq', 'JA1C'], dtype=object)

In [10]:
df = df.fillna(df.mean())

In [11]:
c = []
for i in range(s-39, s+ 1):
    value = 0
    value = 'Var{}'.format(i)
    if value in df.columns:
        c.append(value)

In [12]:
def top_labels_train(X_train, col, max_val):
    top_all = []
    top = list(X_train[col].value_counts().sort_values(ascending=False).head(max_val).index)
    return X_train.apply(lambda x: np.NaN if x[col] not in top else x[col], axis=1)

In [13]:
for x in c:
    df[x] = top_labels_train(df, x, 10)

In [14]:
df = df.dropna(axis=1)

In [15]:
'''for i in range(s-39, s+ 1):
    value = 0
    value = 'Var{}'.format(i)
    try:
        df.drop(value, axis=1, inplace=True)
    except:
        pass'''

"for i in range(s-39, s+ 1):\n    value = 0\n    value = 'Var{}'.format(i)\n    try:\n        df.drop(value, axis=1, inplace=True)\n    except:\n        pass"

In [16]:
df

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var187,Var188,Var189,Var190,Var196,Var210,Var211,Var221,Var227,churn
0,11.487179,0.004029,425.298387,0.125396,238793.32885,1526.000000,7.000000,48.145299,392605.656355,8.625806,...,16.54416,167.368477,462.000000,22007.045192,1K8T,uKAI,L84s,oslk,RAYp,-1
1,11.487179,0.004029,425.298387,0.125396,238793.32885,525.000000,0.000000,48.145299,392605.656355,8.625806,...,16.54416,167.368477,270.142137,22007.045192,1K8T,uKAI,L84s,oslk,RAYp,1
2,11.487179,0.004029,425.298387,0.125396,238793.32885,5236.000000,7.000000,48.145299,392605.656355,8.625806,...,16.54416,167.368477,270.142137,22007.045192,1K8T,uKAI,L84s,Al6ZaUT,02N6s8f,-1
3,11.487179,0.004029,425.298387,0.125396,238793.32885,1326.437116,0.000000,48.145299,392605.656355,8.625806,...,16.54416,167.368477,270.142137,22007.045192,1K8T,uKAI,Mtgm,oslk,RAYp,-1
4,11.487179,0.004029,425.298387,0.125396,238793.32885,1029.000000,7.000000,48.145299,392605.656355,8.625806,...,16.54416,167.368477,270.142137,22007.045192,1K8T,uKAI,L84s,oslk,RAYp,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,11.487179,0.004029,425.298387,0.125396,238793.32885,357.000000,0.000000,48.145299,392605.656355,8.625806,...,16.54416,167.368477,270.142137,22007.045192,1K8T,uKAI,L84s,oslk,RAYp,-1
49996,11.487179,0.004029,425.298387,0.125396,238793.32885,1078.000000,0.000000,48.145299,392605.656355,8.625806,...,16.54416,167.368477,396.000000,22007.045192,1K8T,uKAI,Mtgm,oslk,RAYp,-1
49997,11.487179,0.004029,425.298387,0.125396,238793.32885,2807.000000,7.000000,48.145299,392605.656355,8.625806,...,16.54416,167.368477,270.142137,22007.045192,1K8T,uKAI,L84s,oslk,RAYp,-1
49998,11.487179,0.004029,425.298387,0.000000,238793.32885,1326.437116,6.809496,48.145299,392605.656355,8.625806,...,16.54416,167.368477,276.000000,22007.045192,1K8T,uKAI,Mtgm,oslk,RAYp,-1


In [17]:
X_train, X_test = train_test_split(df ,test_size=0.2)


In [18]:
c = []
for i in range(s-39, s+ 1):
    value = 0
    value = 'Var{}'.format(i)
    if value in df.columns:
        c.append(value)

In [19]:
df_churn = X_train[X_train['churn']==1]
df_not = X_train[X_train['churn']==-1]
df_upsampled = resample(df_churn, replace=True, n_samples=df_not.shape[0])

X_train = pd.concat([df_not, df_upsampled])

In [20]:
y_train = X_train['churn']
X_train = X_train.drop(['churn'],axis=1)


In [21]:
y_test = X_test['churn']
X_test = X_test.drop(['churn'],axis=1)


In [22]:
X_train = X_train.reset_index()

In [23]:
X_test = X_test.reset_index()

In [24]:
ohe1 = OneHotEncoder()
codes = ohe1.fit_transform(X_train[c]).toarray()
feature_names = ohe1.get_feature_names(c)
#X_test[c]= OneHotEncoder().fit_transform(X_test[c])

In [25]:
X_train = pd.concat([X_train.drop(c, axis=1), pd.DataFrame(codes, columns=feature_names)], axis=1)

In [26]:
ohe2 = OneHotEncoder()
codes2 = ohe2.fit_transform(X_test[c]).toarray()
feature_names2 = ohe2.get_feature_names(c)

In [27]:
X_test = pd.concat([X_test.drop(c, axis=1), pd.DataFrame(codes2, columns=feature_names2)], axis= 1)

In [28]:
model = LogisticRegression(solver='liblinear',max_iter=200, verbose=10 )
model.fit(X_train, y_train)

[LibLinear]

c:\Users\munch\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LogisticRegression(max_iter=200, solver='liblinear', verbose=10)

In [33]:
X_train.columns

Index(['index', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var9',
       'Var10',
       ...
       'Var221_oslk', 'Var221_z4pH', 'Var221_zCkv', 'Var227_02N6s8f',
       'Var227_6fzt', 'Var227_RAYp', 'Var227_ZI9m', 'Var227_nIGXDli',
       'Var227_nIGjgSB', 'Var227_vJ_w8kB'],
      dtype='object', length=201)

In [35]:
list(set(X_train.columns) - set(X_test.columns))

['Var196_mKeq']

In [37]:
X_test['Var196_mKeq'] = 0

In [38]:
y_pred = model.predict(X_test)

In [40]:
roc_auc_score(y_test, y_pred)

0.5948542669175747